# Artificial Intelligence CA1
## *Genetic Algorithm*
## Arian Firoozi 810100196

In this assignment curve fitting problem is solved using genetic algorithm.

## 1. Basic Concepts:
Each chromosome contains a list of genes, each representing one coefficient of an nth-degree equation. The first gene represents the coefficient of x^0 and the last gene is the coefficient of x^n. Obviously, value of the last gene cannot be zero since it would make the equation from the order of n - 1.

All of the code is implemented in the class `GeneticAlg`. Initial values of the class that should be given by user are as follows:

* `population_size`: size of the population pool. This number must be even, otherwise, it will be increased by one to fit the standard
* `cycles`: number of generations that will be created in order to solve the problem
* `equation_degree`
* `max_coefficient`
* `min_coefficient`
* `mutate_prob`: probability of mutation of a chromosome
* `mutate_gene_prob`: probability of mutation of a chromosome
* `carry_prob`: percentage of best chromosomes that will be carried to the next generation
* `crossover_prob`: probability of crossover happening between two chromosomes
* `points`: points given by user


## 2. Initial Population
     The `init_pool` method generates random chromosomes with genes ranging from minimum value of coefficients to maximum value of it, which is given to the class as input.

## 3. Fitnees Function
    The `fitness` methods generates a float number ranging from 0 to 1, displaying how close the points are to the formula generated by a certain chromosome (which is shown by chromosome(x) in below formula)
    
$$fitness = \frac{1}{1 + \Sigma|chromosome(x) - y|}$$

## 4. Generation of New Population
    For generating a new population, first `carry_pool` is created. This list contains a number of best chromosomes that will be carried to next generation without crossover or mutation. Then the next population will be generated by creating mating and crossover pools, then pool will be combined with carry pool and lastly, some chromosomes will be mutated to create new generation.

### Mating
    Mating pool is generated by `create_mating_pool` method using random selection. Biased random selection for mating pool has been tested, but considering that a carried pool would already be added to the pool, it proved to not be useful because of the nature of curve fitting problems.

### Crossover
    Crossover pool is generated by `create_crossover_pool` method by first implementing crossover on mating pool and then cutting off the lowest fit chromosomes to fit the size of population. This included earlier generation to get best results from both populations at first, but as the carry system is implemented it no longer needs the previous population since best chromosomes will be carried to the population anyway.

### Mutate
    Mutation pool is generated by `create_mutate_pool` method which uses a biased random mutation. Individuals are selected randomly, but genes mutate in a biased random method which mutates lower-degree coefficients more often than higher-degree ones. Since high degree coefficients play a greater role in determining fitness function and tend to converge earlier than other coefficents and almost in all scenarios mutating these genes produces worse results (especially when algorithm runs on functions of degree 5 or higher). Please note that mutating rate is 50% in test runs but that doesn't mean genes mutate too much as explained above. Here is a plotted comparison between biased and random mutation:

![simple](simple.gif)
![biased](biased.gif)

## 5. Genetic Algorithm
    Algorithm is implemented below. User must pass required parameters (mentioned in part 1) to the class and run the code.

In [41]:
import random


class GeneticAlg:
    def __init__(self, population_size, cycles, equation_degree, max_coefficient, min_coefficient,
                 mutate_prob, mutate_gene_prob, carry_prob, crossover_prob, points: list[list]):

        self.population_size = population_size + population_size % 2
        self.cycles = cycles
        self.equation_degree = equation_degree
        self.max_coefficient = max_coefficient
        self.min_coefficient = min_coefficient
        self.mutate_prob = mutate_prob
        self.mutate_gene_prob = mutate_gene_prob
        self.carry_prob = carry_prob
        self.crossover_prob = crossover_prob
        self.points = points

        self.carry_size = (int(self.carry_prob * self.population_size) // 2) * 2
        self.answer = None

    def create_gene(self) -> int:
        return random.randint(self.min_coefficient, self.max_coefficient)

    def create_gene_nonzero(self) -> int:
        gene = self.create_gene()
        while gene == 0:
            gene = self.create_gene()
        return gene

    def create_individual(self) -> list:
        return [self.create_gene() for _ in range(self.equation_degree)] + \
               [self.create_gene_nonzero()]

    def init_pool(self) -> list:
        return [self.create_individual() for _ in range(self.population_size)]

    def calc_y(self, individual: list, x: int) -> int:
        return sum([x ** power * coeff for power, coeff in enumerate(individual)])

    def y_distance(self, points: list, individual: list):
        return sum([abs(point[1] - self.calc_y(individual, point[0])) for idx, point in enumerate(points)])

    def fitness(self, individual: list) -> float:
        return 1 / (1 + self.y_distance(self.points, individual))

    def get_all_fitness(self, pool: list):
        return [self.fitness(individual) for individual in pool]

    def sorted_pool(self, pool: list):
        return [individual[1] for individual in sorted(zip(self.get_all_fitness(pool), pool))]

    def carry_pool(self, pool: list):
        return self.sorted_pool(pool)[len(pool) - self.carry_size:]

    def create_mating_pool(self, pool: list) -> list:
        mating_pool = pool
        random.shuffle(mating_pool)

        return mating_pool[:self.population_size - self.carry_size]

    def crossover(self, pool: list) -> list:
        for i in range(len(pool), 2):
            if random.random() < self.crossover_prob():
                crossover_point = random.randint(0, self.equation_degree)
                pool[i][:crossover_point], pool[i + 1][:crossover_point] = \
                    pool[i+1][:crossover_point], pool[i][:crossover_point]
        return pool

    def create_crossover_pool(self, pool: list):
        crossover_pool = self.sorted_pool(self.crossover(pool))
        return crossover_pool[:self.population_size - self.carry_size]

    def mutate_individual(self, individual: list):
        def is_mutant(i) -> bool:
            return random.random() * i < self.mutate_gene_prob

        def mutate_nonzero(gene: int,i):
            return [self.create_gene_nonzero() if is_mutant(i) else gene]

        mutated = []
        for i in range(len(individual)-1):
            mutated.append(self.create_gene() if is_mutant(i) else individual[i])
        mutated += mutate_nonzero(individual[-1],len(individual)-1)
        return mutated

    def create_mutation_pool(self, pool: list):
        def is_mutant() -> bool:
            return random.random() < self.mutate_prob

        return [self.mutate_individual(i) if is_mutant() else i for i in pool]

    def print_ans(self):

        def coeff_to_str(coeff):
            if coeff == -1:
                return "-"
            str_coeff = "+" if coeff > 0 else ""
            str_coeff += str(coeff) if coeff != 1 else ""
            return str_coeff

        def individual_to_string(individual: list) -> str:
            print_line = "y = "
            for i in range(len(individual) - 1, 0, -1):
                if individual[i]:
                    coeff_str = coeff_to_str(individual[i])
                    if i == len(individual) - 1 and individual[-1] > 0:
                        coeff_str = coeff_str[1:]
                    print_line += coeff_str + "x^" + str(i) + " "
            print_line += ("+" if individual[0] > 0 else "") + str(individual[0])

            return print_line

        if self.fitness(self.answer) == 1:
            print(individual_to_string(self.answer))
        else:
            print("No such equation found! Best chromosome is:", individual_to_string(self.answer))

    def get_ans(self):
        return self.answer

    def run(self):
        pool = self.init_pool()
        best_ans = None
        for _ in range(self.cycles):
            carry_pool = self.carry_pool(pool)
            pool = self.create_mating_pool(pool)
            pool = self.create_crossover_pool(pool)
            pool += carry_pool
            pool = self.create_mutation_pool(pool)

            all_fitness = self.get_all_fitness(pool)
            max_fitness = max(all_fitness)
            best_indiv = pool[all_fitness.index(max_fitness)]
            if best_ans is None or self.fitness(best_ans) < max_fitness:
                best_ans = best_indiv
            if max_fitness == 1:
                self.answer = best_indiv
                break
        if self.answer is None:
            self.answer = best_ans


here is an example run of the algorithm:

In [42]:
test = GeneticAlg(population_size=100,
                 cycles=10000,
                 equation_degree=3,
                 max_coefficient=10,
                 min_coefficient=-10,
                 mutate_prob=0.5,
                 mutate_gene_prob=0.5,
                 carry_prob=0.1,
                 crossover_prob=0.6,
                 points = [[0, 1], [1, 0], [2, -5], [-1, -8]])
test.run()
test.print_ans()


y = x^3 -5x^2 +3x^1 +1


For harder problems, a faster approach is to use the multi-start version:

In [44]:
multi_start = 10

for i in range(multi_start):
    multi_test = GeneticAlg(population_size=100,
                            cycles=1000,
                            equation_degree=5,
                            max_coefficient=10,
                            min_coefficient=-10,
                            mutate_prob=0.5,
                            mutate_gene_prob=0.5,
                            carry_prob=0.1,
                            crossover_prob=0.6,
                            points=[[1,2],[3,-990],[2,-87],[-3,2418],[0,3]])
    
    multi_test.run()
    if multi_test.fitness(multi_test.answer) == 1 or i == multi_start - 1:
        multi_test.print_ans()
        break


y = -7x^5 +9x^4 -2x^2 -x^1 +3


## 6. Questions
1. If the population is too large it can slow the algorithm down since time complexity is not linear, while a significantly small population may fail to find the answer since the diversity of the chromosome pool is not enough.
2. If the previous population remains, each generation population will grow exponentially and after a few generations, the algorithm will require large memory resources and will be extremely slow. If not all previous generation remains, it can be handled in lower generation cycles, however, it could be troublesome as the generation cycles increase.
3. Crossover can help obtain new chromosomes that combine best traits of their parents, while mutation can generate better or worse chromosomes. Some of these **better** chromosomes can be vital to finding the answer since not all problems have greedy answers. Using only crossover can lead to getting stuck in local optimums while relying only on mutation for making better chromosomes will randomize the algorithm, ultimately defeating the purpose of genetic algorithm.
4. We can use multi-objective optimization method. A multistart method can also be implemented to avoid local optimums.
5. Lack of diversity can lead to getting stuck in local optimums. Increasing the mutation rate and avoiding selection of same parents for new chromosomes can solve this problem.
6. Using limited cycles of generation (which this algorithm already does), or setting a runtime limit for the algorithm.
7. Generally, higher-degree problems tend to be harder to solve since the size of chromosomes will increase and time complexity is usually n^2 or higher.
8. More information about the curve helps to find a certain curve more easily and it will increase search time. Fewer points in space can define more than one curve which is typically easier to solve and may give us multiple answers. 